<a href="https://colab.research.google.com/github/sfarias03/Education-Inequality/blob/main/SF_Education_Inequality_Data_Preparation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Introduction

This research project aims to address unequal access to education in American high schools by analyzing the performance of students in the ACT or SAT examinations. We will utilize the EdGap dataset from EdGap.org, which contains 2016 data on the average ACT or SAT scores of schools and socio-economic indicators of school districts, and supplement it with information from the National Center for Education Statistics. Through the use of Python and data visualization tools, we aim to uncover the factors contributing to unequal educational outcomes.

## Import libraries

In [209]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_theme(style='whitegrid')
import missingno as msno

# Train-test splits
from sklearn.model_selection import train_test_split

# Model preprocessing
from sklearn.preprocessing import StandardScaler

# Imputation
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import SimpleImputer, IterativeImputer, KNNImputer


# Model preprocessing
from sklearn import preprocessing
from sklearn.preprocessing import PolynomialFeatures
from sklearn.preprocessing import StandardScaler

# Train-test splits
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV, KFold, RandomizedSearchCV

# Models
from sklearn import linear_model


# Model metrics and analysis
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error

### EdGap data


The Census Bureau's American Community Survey provides all socio-economic data, including household income, unemployment, adult educational attainment, and family structure. [EdGap.org](https://www.edgap.org/#5/37.875/-96.987) reports that ACT and SAT score data is sourced from each state's department of education or another public data release. Although the nature of the other public data release is unknown, the quality of the census data and department of education data is assumed to be reasonably high. While [EdGap.org](https://www.edgap.org/#5/37.875/-96.987) does not indicate any processing of the data, there is a possibility of human error since the data was assembled by the [EdGap.org](https://www.edgap.org/#5/37.875/-96.987) team. However, given the public nature of the data, original data sources can be consulted to verify the data's quality if necessary.

### School information data

[National Center for Education Statistics](https://nces.ed.gov/ccd/pubschuniv.asp) is the source of the school information data, which contains basic identifying information about schools and is assumed to be of reasonably high quality. 


## Load the data

In [210]:
edgap = pd.read_excel('https://raw.githubusercontent.com/brian-fischer/DATA-3320/main/education/EdGap_data.xlsx')

/usr/local/lib/python3.9/dist-packages/openpyxl/worksheet/_reader.py:312: UserWarning: Unknown extension is not supported and will be removed
  warn(msg)


In [211]:
!wget https://www.dropbox.com/s/lkl5nvcdmwyoban/ccd_sch_029_1617_w_1a_11212017.csv?dl=0

--2023-04-25 00:15:17--  https://www.dropbox.com/s/lkl5nvcdmwyoban/ccd_sch_029_1617_w_1a_11212017.csv?dl=0
Resolving www.dropbox.com (www.dropbox.com)... 162.125.3.18, 2620:100:6018:18::a27d:312
Connecting to www.dropbox.com (www.dropbox.com)|162.125.3.18|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: /s/raw/lkl5nvcdmwyoban/ccd_sch_029_1617_w_1a_11212017.csv [following]
--2023-04-25 00:15:18--  https://www.dropbox.com/s/raw/lkl5nvcdmwyoban/ccd_sch_029_1617_w_1a_11212017.csv
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://uc8a8e0cb35ef2a1e1d71327f60b.dl.dropboxusercontent.com/cd/0/inline/B60ENwiLb5Iq4wKhn0N8kcwZrQxVU4TTIbAWE2i9_vTJW7baQB5uOdYdGEltXtUjeaDO5hqWdWG5hY1k1wYY4I4XlLSrpmSYFg6nJLiFey7S0mmNWTls5VWvWaS4fUiE1oGRrFq_EDYlxDLsqDSmA1ZcyBUhRjIUwqQmcOQQrhPUWg/file# [following]
--2023-04-25 00:15:18--  https://uc8a8e0cb35ef2a1e1d71327f60b.dl.dropboxusercontent.com/cd/0/inline/B60ENwiLb

In [212]:
school_info = pd.read_csv('ccd_sch_029_1617_w_1a_11212017.csv?dl=0', encoding='unicode_escape')

<ipython-input-212-da1a7837c049>:1: DtypeWarning: Columns (6,9,14,15,18,19,21,22,25,26,29,31,35,39,41,42) have mixed types. Specify dtype option on import or set low_memory=False.
  school_info = pd.read_csv('ccd_sch_029_1617_w_1a_11212017.csv?dl=0', encoding='unicode_escape')


## Explore the contents of the data sets

In [213]:
edgap.head()

,NCESSCH School ID,CT Unemployment Rate,CT Pct Adults with College Degree,CT Pct Childre In Married Couple Family,CT Median Household Income,School ACT average (or equivalent if SAT score),School Pct Free and Reduced Lunch
0,100001600143,0.117962,0.445283,0.346495,42820.0,20.433455,0.066901
1,100008000024,0.063984,0.662765,0.767619,89320.0,19.498168,0.112412
2,100008000225,0.056460,0.701864,0.713090,84140.0,19.554335,0.096816
3,100017000029,0.044739,0.692062,0.641283,56500.0,17.737485,0.296960
4,100018000040,0.077014,0.640060,0.834402,54015.0,18.245421,0.262641


In [214]:
school_info.head()

,SCHOOL_YEAR,FIPST,STATENAME,ST,SCH_NAME,LEA_NAME,STATE_AGENCY_NO,UNION,ST_LEAID,LEAID,...,G_10_OFFERED,G_11_OFFERED,G_12_OFFERED,G_13_OFFERED,G_UG_OFFERED,G_AE_OFFERED,GSLO,GSHI,LEVEL,IGOFFERED
0,2016-2017,1,ALABAMA,AL,Sequoyah Sch - Chalkville Campus,Alabama Youth Services,1,NaN,AL-210,100002,...,Yes,Yes,Yes,No,No,No,07,12,High,As reported
1,2016-2017,1,ALABAMA,AL,Camps,Alabama Youth Services,1,NaN,AL-210,100002,...,Yes,Yes,Yes,No,No,No,07,12,High,As reported
2,2016-2017,1,ALABAMA,AL,Det Ctr,Alabama Youth Services,1,NaN,AL-210,100002,...,Yes,Yes,Yes,No,No,No,07,12,High,As reported
3,2016-2017,1,ALABAMA,AL,Wallace Sch - Mt Meigs Campus,Alabama Youth Services,1,NaN,AL-210,100002,...,Yes,Yes,Yes,No,No,No,07,12,High,As reported
4,2016-2017,1,ALABAMA,AL,McNeel Sch - Vacca Campus,Alabama Youth Services,1,NaN,AL-210,100002,...,Yes,Yes,Yes,No,No,No,07,12,High,As reported


## Convert data types, if necessary

As we plan for the next step, we aim to merge the DataFrames using the school's identity as the key, identified by the NCESSCH school identity. However, the two DataFrames refer to this identity by different names and formats, with the EdGap data set using int64 and the school information data set using float64. To address this, we will convert the NCESSCH column in the school_info DataFrame to int64. It is important to note that we will need to drop rows where NCESSCH is a NaN value before performing the conversion.

In [215]:
school_info = school_info[school_info['NCESSCH'].isna() == False]

In [216]:
school_info['NCESSCH'] = school_info['NCESSCH'].astype('int64')

<ipython-input-216-006ea9997674>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  school_info['NCESSCH'] = school_info['NCESSCH'].astype('int64')


Following this, we will proceed to select specific portions of the data that are relevant to our analysis.

## Select relevant subsets of the data

Since the school information data set contains a lot of information. We only need the year, school identity, location, and school type information.



In [217]:
school_info = school_info[['SCHOOL_YEAR', 'NCESSCH', 'MSTATE', 'MZIP', 'SCH_TYPE_TEXT', 'LEVEL']]

## Rename columns

In [218]:
edgap = edgap.rename(columns={"NCESSCH School ID":"id", 
              "CT Pct Adults with College Degree":"percent_college",        
              "CT Unemployment Rate":"rate_unemployment", 
              "CT Pct Childre In Married Couple Family":"percent_married",
              "CT Median Household Income":"median_income",
              "School ACT average (or equivalent if SAT score)":"average_act",
              "School Pct Free and Reduced Lunch":"percent_lunch"})

In [219]:
school_info = school_info.rename(columns={'SCHOOL_YEAR':'year', 
                                          'NCESSCH':'id', 
                                          'MSTATE':'state',
                                          'MZIP':'zip_code',
                                          'SCH_TYPE_TEXT':'school_type',
                                          'LEVEL':'school_level'})

## Join data frames 

In [220]:
df = edgap.merge(school_info, how = 'left', on='id')

## Quality Control

Next we will check for out-of-range values or values that do not match what we want to analyze. 

In [221]:
df.describe()

,id,rate_unemployment,percent_college,percent_married,median_income,average_act,percent_lunch
count,7.986000e+03,7972.000000,7973.000000,7961.000000,7966.000000,7986.000000,7986.000000
mean,3.321869e+11,0.098730,0.568930,0.633440,52026.905222,20.181532,0.420651
std,1.323638e+11,0.058959,0.165704,0.196764,24228.057079,2.595201,0.239754
min,1.000016e+11,0.000000,0.091493,0.000000,3589.000000,-3.070818,-0.054545
25%,2.105340e+11,0.058655,0.450828,0.523810,36597.250000,18.600000,0.238501
50%,3.600085e+11,0.085649,0.554979,0.667594,46833.500000,20.400000,0.381570
75%,4.226678e+11,0.123376,0.676571,0.777135,61369.250000,21.910867,0.575447
max,5.605830e+11,0.590278,1.000000,1.000000,226181.000000,32.362637,0.998729


In [222]:
df.agg(['min', 'max']).round(2)

<ipython-input-222-39a2038683d4>:1: FutureWarning: ['year', 'state', 'zip_code', 'school_type', 'school_level'] did not aggregate successfully. If any error is raised this will raise in a future version of pandas. Drop these columns/ops to avoid this warning.
  df.agg(['min', 'max']).round(2)


,id,rate_unemployment,percent_college,percent_married,median_income,average_act,percent_lunch
min,100001600143,0.00,0.09,0.0,3589.0,-3.07,-0.05
max,560583000335,0.59,1.00,1.0,226181.0,32.36,1.00


In [223]:
df['state'].unique()

array(['DE', 'FL', 'GA', nan, 'IL', 'IN', 'KY', 'LA', 'MA', 'MI', 'MO',
       'NJ', 'NY', 'NC', 'OH', 'PA', 'TN', 'TX', 'WA', 'WI', 'WY'],
      dtype=object)

In [224]:
df.loc[df['percent_lunch'] < 0, 'percent_lunch'] = np.nan

In [225]:
df = df.drop(df.loc[df['average_act'] <=0].index)

We will only be checking High schools.

In [226]:
df.loc[df['school_level']== 'High']

,id,rate_unemployment,percent_college,percent_married,median_income,average_act,percent_lunch,year,state,zip_code,school_type,school_level
0,100001600143,0.117962,0.445283,0.346495,42820.0,20.433455,0.066901,2016-2017,DE,19804,Regular School,High
1,100008000024,0.063984,0.662765,0.767619,89320.0,19.498168,0.112412,2016-2017,DE,19709,Regular School,High
2,100008000225,0.056460,0.701864,0.713090,84140.0,19.554335,0.096816,2016-2017,DE,19709,Regular School,High
3,100017000029,0.044739,0.692062,0.641283,56500.0,17.737485,0.296960,2016-2017,DE,19958,Regular School,High
4,100018000040,0.077014,0.640060,0.834402,54015.0,18.245421,0.262641,2016-2017,DE,19934,Regular School,High
...,...,...,...,...,...,...,...,...,...,...,...,...
7981,560530200294,0.034549,0.590780,0.720077,64891.0,18.500000,0.232156,2016-2017,WY,82902,Regular School,High
7982,560569000311,0.069538,0.633860,0.808841,44603.0,22.300000,0.083871,2016-2017,WY,82833,Regular School,High
7983,560569500360,0.035159,0.764566,0.776570,44896.0,21.500000,0.184829,2016-2017,WY,82801,Regular School,High
7984,560576200324,0.063877,0.670532,0.879324,92134.0,19.700000,0.112583,2016-2017,WY,82935,Regular School,High


## Identify missing values

In [227]:
df.isna().sum()

id                    0
rate_unemployment    14
percent_college      13
percent_married      25
median_income        20
average_act           0
percent_lunch        20
year                 88
state                88
zip_code             88
school_type          88
school_level         88
dtype: int64

In [228]:
df = df.loc[df['average_act'].isna() == False]

Some schools are missing all four socioeconomic variables, but most of them lack only a portion of the data. Dropping the rows with NaN values would harm our analysis of the variables with available data. Thus, we will retain the rows with missing socioeconomic variables. However, we will eliminate the rows without the average ACT score. After splitting the data for model evaluation into training and testing sets, we will fill in the remaining missing values using imputation.

## Train test split

In [229]:
X = df[df.columns.difference(['id', 'average_act'])]

In [230]:
Y = df[['average_act']]

Next, we will define the predictor variable matrix X to include all columns except for id and average_act, and define the output variable y to be average_act.

We will utilize the test set approach to evaluate models that predict the average_act.

In [231]:
x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size=.2, random_state = 1)

In [232]:
print(x_train.shape, x_test.shape)

(6386, 10) (1597, 10)


Verify that the means and standard deviations of the training and testing input variables are comparable.

In [233]:
x_train.agg(['mean','std']).round(2)

<ipython-input-233-4fb165f81f78>:1: FutureWarning: ['school_level', 'school_type', 'state', 'year', 'zip_code'] did not aggregate successfully. If any error is raised this will raise in a future version of pandas. Drop these columns/ops to avoid this warning.
  x_train.agg(['mean','std']).round(2)


,median_income,percent_college,percent_lunch,percent_married,rate_unemployment
mean,52094.78,0.57,0.42,0.63,0.10
std,24319.92,0.17,0.24,0.20,0.06


In [234]:
x_test.agg(['mean','std']).round(2)

<ipython-input-234-ecad6fcf8116>:1: FutureWarning: ['school_level', 'school_type', 'state', 'year', 'zip_code'] did not aggregate successfully. If any error is raised this will raise in a future version of pandas. Drop these columns/ops to avoid this warning.
  x_test.agg(['mean','std']).round(2)


,median_income,percent_college,percent_lunch,percent_married,rate_unemployment
mean,51802.85,0.57,0.42,0.63,0.10
std,23860.56,0.16,0.23,0.19,0.06


## Data imputation

In [235]:
imputer = IterativeImputer()

Impute the missing values in the training data.

In [236]:
imputer.fit(x_train.loc[:,'median_income' : 'rate_unemployment'])

IterativeImputer()

In [237]:
x_train.loc[:, 'median_income':'rate_unemployment'] = imputer.transform(x_train.loc[:, 'median_income':'rate_unemployment'])

In [238]:
x_train.isna().sum()

median_income         0
percent_college       0
percent_lunch         0
percent_married       0
rate_unemployment     0
school_level         71
school_type          71
state                71
year                 71
zip_code             71
dtype: int64

In [239]:
x_test.isna().sum()

median_income         5
percent_college       4
percent_lunch         5
percent_married       8
rate_unemployment     4
school_level         17
school_type          17
state                17
year                 17
zip_code             17
dtype: int64

In [240]:
df_train = x_train.join(y_train)

In [241]:
df_train.head()

,median_income,percent_college,percent_lunch,percent_married,rate_unemployment,school_level,school_type,state,year,zip_code,average_act
7626,48514.0,0.518152,0.253521,0.697964,0.069390,High,Regular School,WI,2016-2017,54112,21.000000
5436,26012.0,0.548486,0.000000,0.055385,0.072592,High,Regular School,PA,2016-2017,17101,20.665446
1732,58050.0,0.653129,0.215569,0.834146,0.108871,High,Regular School,IN,2016-2017,46064,21.791209
3186,55303.0,0.825795,0.923828,1.000000,0.042091,High,Regular School,MO,2016-2017,63103,15.600000
1913,39814.0,0.618495,0.693817,0.572973,0.110555,High,Regular School,KY,2016-2017,40214,15.700000


In [242]:
df_train.describe()

,median_income,percent_college,percent_lunch,percent_married,rate_unemployment,average_act
count,6386.000000,6386.000000,6386.000000,6386.000000,6386.000000,6386.000000
mean,52081.670430,0.569524,0.421765,0.633034,0.098654,20.200396
std,24305.896428,0.166607,0.239766,0.197487,0.058645,2.555798
min,2435.408369,0.093122,0.000000,0.000000,0.000000,12.500000
25%,36481.000000,0.451024,0.238643,0.524383,0.058884,18.614164
50%,46796.000000,0.555995,0.381871,0.666362,0.085456,20.400000
75%,61512.000000,0.677627,0.573638,0.777510,0.123265,21.935287
max,226181.000000,1.000000,0.998532,1.000000,0.576125,30.946276


In [243]:
df_test = x_test.join(y_test)

## Export the clean .csv files

Export files with the training and testing data

In [245]:
from google.colab import files

df_train.to_csv('clean_education_inequality.csv', encoding = 'utf-8-sig', index=False) 

files.download('clean_education_inequality.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>